# Predicting NFL Game Outcomes
## Rohan Lewis, Michael Nercessian, Matthew Armstrong

# 2019 NFL Game by Game Data

We first manually downloaded our data from this source: https://www.pro-football-reference.com/teams/

Our first step will be to import all of the modules we need, and then load the data.

In [81]:
import Pkg
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("DataFrames")
Pkg.add("Statistics")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
using Random
using CSV
using Plots
using DataFrames
using Statistics

In [23]:
nfl_teams = CSV.read("Datasets/nfl_teams.csv")
nfl_teams_with_spaces = CSV.read("Datasets/nfl_teams_with_spaces.csv")
showall(nfl_teams_with_spaces)

32×5 DataFrame
│ Row │ ID    │ Name                 │ Abbreviation │ Conference │ Division │
│     │ Int64 │ String               │ String       │ String     │ String   │
├─────┼───────┼──────────────────────┼──────────────┼────────────┼──────────┤
│ 1   │ 1     │ Arizona Cardinals    │ ARI          │ NFC        │ West     │
│ 2   │ 2     │ Atlanta Falcons      │ ATL          │ NFC        │ South    │
│ 3   │ 3     │ Baltimore Ravens     │ BAL          │ AFC        │ North    │
│ 4   │ 4     │ Buffalo Bills        │ BUF          │ AFC        │ East     │
│ 5   │ 5     │ Carolina Panthers    │ CAR          │ NFC        │ South    │
│ 6   │ 6     │ Chicago Bears        │ CHI          │ NFC        │ North    │
│ 7   │ 7     │ Cincinnati Bengals   │ CIN          │ AFC        │ North    │
│ 8   │ 8     │ Cleveland Browns     │ CLE          │ AFC        │ North    │
│ 9   │ 9     │ Dallas Cowboys       │ DAL          │ NFC        │ East     │
│ 10  │ 10    │ Denver Broncos       │ DEN       

In [24]:
abbreviations = [nfl_teams[:, 1] nfl_teams[:, 3]]

32×2 Array{Any,2}:
  1  "ARI"
  2  "ATL"
  3  "BAL"
  4  "BUF"
  5  "CAR"
  6  "CHI"
  7  "CIN"
  8  "CLE"
  9  "DAL"
 10  "DEN"
 11  "DET"
 12  "GB"
 13  "HOU"
  ⋮  
 21  "NYG"
 22  "NYJ"
 23  "LV"
 24  "PHI"
 25  "PIT"
 26  "LAC"
 27  "SF"
 28  "SEA"
 29  "LAR"
 30  "TB"
 31  "TEN"
 32  "WAS"

In [25]:
team_names = nfl_teams[:, 2]
team_names_split = String.(reduce(hcat,split.(team_names,' ')))
team_names_split = team_names_split[2, :]
team_names_split = [nfl_teams[:, 1] map(lowercase, team_names_split)]

32×2 Array{Any,2}:
  1  "cardinals"
  2  "falcons"
  3  "ravens"
  4  "bills"
  5  "panthers"
  6  "bears"
  7  "bengals"
  8  "browns"
  9  "cowboys"
 10  "broncos"
 11  "lions"
 12  "packers"
 13  "texans"
  ⋮  
 21  "giants"
 22  "jets"
 23  "raiders"
 24  "eagles"
 25  "steelers"
 26  "chargers"
 27  "49ers"
 28  "seahawks"
 29  "rams"
 30  "buccaneers"
 31  "titans"
 32  "redskins"

In [26]:
team_paths = []
for i in 1:size(team_names_split,1)
    team_names_split[i,2] = "Datasets/2019 Teams/" * team_names_split[i,2] * "_2019.csv"
end
team_names_split

32×2 Array{Any,2}:
  1  "Datasets/2019 Teams/cardinals_2019.csv"
  2  "Datasets/2019 Teams/falcons_2019.csv"
  3  "Datasets/2019 Teams/ravens_2019.csv"
  4  "Datasets/2019 Teams/bills_2019.csv"
  5  "Datasets/2019 Teams/panthers_2019.csv"
  6  "Datasets/2019 Teams/bears_2019.csv"
  7  "Datasets/2019 Teams/bengals_2019.csv"
  8  "Datasets/2019 Teams/browns_2019.csv"
  9  "Datasets/2019 Teams/cowboys_2019.csv"
 10  "Datasets/2019 Teams/broncos_2019.csv"
 11  "Datasets/2019 Teams/lions_2019.csv"
 12  "Datasets/2019 Teams/packers_2019.csv"
 13  "Datasets/2019 Teams/texans_2019.csv"
  ⋮  
 21  "Datasets/2019 Teams/giants_2019.csv"
 22  "Datasets/2019 Teams/jets_2019.csv"
 23  "Datasets/2019 Teams/raiders_2019.csv"
 24  "Datasets/2019 Teams/eagles_2019.csv"
 25  "Datasets/2019 Teams/steelers_2019.csv"
 26  "Datasets/2019 Teams/chargers_2019.csv"
 27  "Datasets/2019 Teams/49ers_2019.csv"
 28  "Datasets/2019 Teams/seahawks_2019.csv"
 29  "Datasets/2019 Teams/rams_2019.csv"
 30  "Datasets/2019 

In [27]:
# # now let's create some DataFrame variables
# cardinals_19 = CSV.read(team_names_split[1,2])
# falcons_19 = CSV.read(team_names_split[2,2])
# ravens_19 = CSV.read(team_names_split[3,2])
# bills_19 = CSV.read(team_names_split[4,2])
# panthers_19 = CSV.read(team_names_split[5,2])
# bears_19 = CSV.read(team_names_split[6,2])
# bengals_19 = CSV.read(team_names_split[7,2])
# browns_19 = CSV.read(team_names_split[8,2])
# cowboys_19 = CSV.read(team_names_split[9,2])
# broncos_19 = CSV.read(team_names_split[10,2])
# lions_19 = CSV.read(team_names_split[11,2])
# packers_19 = CSV.read(team_names_split[12,2])
# texans_19 = CSV.read(team_names_split[13,2])
# colts_19 = CSV.read(team_names_split[14,2])
# jaguars_19 = CSV.read(team_names_split[15,2])
# chiefs_19 = CSV.read(team_names_split[16,2])
# dolphins_19 = CSV.read(team_names_split[17,2])
# vikings_19 = CSV.read(team_names_split[18,2])
# patriots_19 = CSV.read(team_names_split[19,2])
# saints_19 = CSV.read(team_names_split[20,2])
# giants_19 = CSV.read(team_names_split[21,2])
# jets_19 = CSV.read(team_names_split[22,2])
# raiders_19 = CSV.read(team_names_split[23,2])
# eagles_19 = CSV.read(team_names_split[24,2])
# steelers_19 = CSV.read(team_names_split[25,2])
# chargers_19 = CSV.read(team_names_split[26,2])
# # note: name different than csv file name
# fortyniners_19 = CSV.read(team_names_split[27,2])
# seahawsks_19 = CSV.read(team_names_split[28,2])
# rams_19 = CSV.read(team_names_split[29,2])
# buccaneers_19 = CSV.read(team_names_split[30,2])
# titans_19 = CSV.read(team_names_split[31,2])
# redskins_19 = CSV.read(team_names_split[32,2])
# teams = []
# # for i in 1:32
# #     teams[i] = CSV.read(team_names_split[i,2])

In [9]:
original_teams = Array{DataFrame,1}(undef,32)
for i in 1:32
#     append!(teams, CSV.read(team_names_split[i,2], header))
    original_teams[i] = CSV.read(team_names_split[i,2], header = 2)
end

In [10]:
teams = copy(original_teams)
for i in 1:32    
    teams[i] = teams[i][1:17,:]
    teams[i] = teams[i][.!(ismissing.(teams[i][!, :Date])), :]
    #look into doing all at once
    teams[i] = teams[i][:, 1:22]
    teams[i] = select!(teams[i], Not(:OT))
    teams[i] = select!(teams[i], Not(:Column5))
    teams[i] = select!(teams[i], Not(:Column6))
    teams[i] = select!(teams[i], Not(:Column4))
end

In [11]:
for i in 1:32    
    insertcols!(teams[i], 6, :Team => trunc.(Int,i*ones(16)))
end

In [12]:
#showall(teams[18])
changeOpp = copy(teams)
for k in 1:32
    opps = []
    for i in 1:16
        opp = changeOpp[k][:Opp][i]
        idx = 0
        for j in 1:32
            if nfl_teams_with_spaces[:,2][j] == opp
                idx = j
            end
        end
        append!(opps, idx)
    end
    insertcols!(changeOpp[k], 7, :Opponent => opps)
    changeOpp[k] = select!(changeOpp[k], Not(:Opp))
end

In [13]:
#pats = copy(teams[12])
home = copy(teams)
for i in 1:32
    home[i][:Column9][Not(ismissing.(home[i][:Column9]))] .= "0"
    home[i][:Column9][ismissing.(home[i][:Column9])] .= "1"
end

In [21]:
season_data = DataFrame(home[1])
for i in 2:32
    final = vcat(final, home[i])
end

In [22]:
final

,Week,Day,Date,Rec,Column9,Team,Opponent,Tm,Opp_1
,Any,String?,String?,Any,String?,Int64,Any,Int64?,Int64?
1,1,Sun,September 8,0-0-1,1,1,11,27,27
2,2,Sun,September 15,0-1-1,0,1,3,17,23
3,3,Sun,September 22,0-2-1,1,1,5,20,38
4,4,Sun,September 29,0-3-1,1,1,28,10,27
5,5,Sun,October 6,1-3-1,0,1,7,26,23
6,6,Sun,October 13,2-3-1,1,1,2,34,33
7,7,Sun,October 20,3-3-1,0,1,21,27,21
8,8,Sun,October 27,3-4-1,0,1,20,9,31
9,9,Thu,October 31,3-5-1,1,1,27,25,28
